In [ ]:
import yaml
from mdsetup import MDSetup
from mdsetup.tools.systemsetup import generate_initial_configuration

## Insert guest molecule in system and equilibrate

In [ ]:
gromacs_setup = MDSetup(
    system_setup="input/setup_C1.yaml",
    simulation_default="input/defaults.yaml",
    simulation_ensemble="input/ensemble.yaml",
    submission_command="qsub",
)

# Define guest molecules (solutes) and their repsective coordinate file
solutes = ["CO2", "O2", "CH4", "N2"]
solutes_coord = [
    "/home/st/st_st/st_ac137577/workspace/DES_simulations/coordinates/co2.pdb",
    "/home/st/st_st/st_ac137577/workspace/DES_simulations/coordinates/o2.gro",
    "/home/st/st_st/st_ac137577/workspace/DES_simulations/coordinates/met.gro",
    "/home/st/st_st/st_ac137577/workspace/DES_simulations/coordinates/n2.gro",
]

# Provide the path to the system where the solutes will be added.
initial_path = f"{gromacs_setup.project_folder}/md_thermo/%s/copy_0/04_nvt/nvt.gro"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = ["em", "nvt", "npt_equilibration", "npt_production"]

# Define the simulation time per ensemble in nano seconds
simulation_times = [10000, 2.0, 2.0, 10.0]

# Provide kwargs that should be passed into the input template directly
input_kwargs = {}

# Define number of copies
copies = 0

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set = 0

In [ ]:
# Ensure that all jobs for all solutes are submitted at once
job_files = [[] for _ in gromacs_setup.system_setup["temperature"]]

for solute, solute_coord in zip(solutes, solutes_coord):
    # Define simulation folder and insert guest molecule
    sim_folder = f"free_energy/{solute}/equilibration"

    initial_systems = []

    for state in gromacs_setup.loop_through_states():
    
        # Define folder with defined state attributes
        state_condition = gromacs_setup.define_state_cond(**state)

        # Build system
        build_folder = (
            f"{gromacs_setup.project_folder}/{sim_folder}/insert_solute/"
            f"{state_condition}"
        )

        # Insert solutes to system
        initial_systems.append(
            generate_initial_configuration(
                destination_folder=build_folder,
                build_template=gromacs_setup.system_setup["paths"]["build_template"],
                software=gromacs_setup.system_setup["software"],
                coordinate_paths=[solute_coord],
                molecules_list=[{"name": solute, "number": 1, "smiles": ""}],
                initial_system=initial_path % state_condition,
                box={},
                submission_command=gromacs_setup.submission_command,
            )
        )

    # Add solute to system
    gromacs_setup.system_molecules.append({"name": solute, "number": 1, "smiles": ""})

    # Prepare simulation
    gromacs_setup.prepare_simulation(
        folder_name=sim_folder,
        ensembles=ensembles,
        simulation_times=simulation_times,
        initial_systems=initial_systems,
        input_kwargs=input_kwargs,
        copies=copies,
        off_set=off_set,
    )

    # Remove it again
    gromacs_setup.system_molecules.pop()

    for j, files in enumerate(gromacs_setup.job_files):
        job_files[j].extend(files)

gromacs_setup.job_files = job_files

2) Submit jobs to cluster

In [ ]:
# Submit the simulations
gromacs_setup.submit_simulation()

## Simulate solvation free energy using the decoupling method

In [ ]:
# Define guest molecules (solutes)
solutes = ["CO2", "O2", "CH4", "N2"]

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = ["em", "nvt", "npt_equilibration", "npt_production"]

# Define the simulation time per ensemble in nano seconds
simulation_times = [10000, 2.0, 20.0, 50.0]

# Define number of copies
copies = 0

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set = 0

# Define free energy settings
with open("input/free_energy.yaml") as f:
    free_energy = yaml.safe_load(f)

free_energy["init_lambda_states"] = " ".join(
    [f"{x:5.2f}" for x in free_energy["combined_lambdas"]]
)
free_energy["vdw_lambdas"] = " ".join(
    [f"{max(x-1,0.0):5.2f}" for x in free_energy["combined_lambdas"]]
)
free_energy["coul_lambdas"] = " ".join(
    [f"{min(x,1.0):5.2f}" for x in free_energy["combined_lambdas"]]
)

# Ensure that all jobs for all solutes and all lambdas are submitted at once
job_files = [[] for _ in gromacs_setup.system_setup["temperature"]]

for solute in solutes:
    initial_systems = [
        (
            f"{gromacs_setup.project_folder}/free_energy/{solute}/equilibration/"
            f"{state_cond}/copy_0/03_npt_production/npt_production.gro"
        )
        for state_cond in gromacs_setup.define_state_cond_list()
    ]

    free_energy["couple_moltype"] = solute

    # Add solute to system
    gromacs_setup.system_molecules.append({"name": solute, "number": 1, "smiles": ""})

    for i, _ in enumerate(free_energy["combined_lambdas"]):
        # Define simulation folder for each lambda
        sim_folder = f"free_energy/{solute}/coupling/lambda_{i}"
        free_energy["init_lambda_state"] = i

        gromacs_setup.prepare_simulation(
            folder_name=sim_folder,
            ensembles=ensembles,
            simulation_times=simulation_times,
            initial_systems=initial_systems,
            input_kwargs={"free_energy": free_energy},
            copies=copies,
            off_set=off_set,
        )

        for j, files in enumerate(gromacs_setup.job_files):
            job_files[j].extend(files)

    # Remove it again
    gromacs_setup.system_molecules.pop()

gromacs_setup.job_files = job_files

2) Submit jobs to cluster

In [ ]:
# Submit the simulations
gromacs_setup.submit_simulation()

## Analyse the solvation free energy

In [ ]:
# Define guest molecules (solutes)
solutes = ["CO2", "O2", "CH4", "N2"]

# Define analysis ensemble
ensemble = "03_npt_production"

# Percentage to discard from beginning of the simulation
time_fraction = 0.0

# Free energy method
method = "MBAR"

# If free energy outputs should be decorrelated (recommended)
decorrelate = True

# Whether coupling or decoupling simulations were performed
coupling = False


In [ ]:
for solute in solutes:
    
    print(f"\nAnalysis for solute: {solute}")
    
    # Define simulation folder and insert guest molecule'
    analysis_folder = f"free_energy/{solute}/coupling"

    # Analyse free solvation energy
    gromacs_setup.analysis_solvation_free_energy(
        analysis_folder=analysis_folder,
        ensemble=ensemble,
        method=method,
        time_fraction=time_fraction,
        decorrelate=decorrelate,
        coupling=coupling,
    )